In [1]:
import numpy as np
import pandas as pd

from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

DATA = 'dataset/loan_one_hot_encoded.csv'

In [2]:
drop_cols = ['loan_created', 'application_id',
#              'firm_type_Proprietorship',
             'average_business_inflow'
            ]
df = pd.read_csv(DATA)
Y = df['loan_created']
og_X = df.drop(drop_cols, axis=1)

Things to-do:

- [ ] The data is missing values; use Imputer to fill mean/median of the column
    - [ ] Create another column to denote whether the data was imputed or not; I've read that it seems to have better results
- [ ] Set class_weights in SVM
- [ ] Tune hyperparameters
- [ ] Specific kernel? 

What to do about skewed data:
- See as an anamoly detection problem?
- class weights (for SVM)
- Remove training data (less data anyway..:( )


In [3]:
imp = Imputer()
imputed_X = imp.fit_transform(og_X)

# X = imputed_X
scl = StandardScaler()
X = scl.fit_transform(imputed_X)

In [4]:
# pd.value_counts(og_X['firm_type_Proprietorship'])

In [5]:
X.shape

(230, 240)

In [6]:
reverse_Y = Y.apply(lambda x: 0 if x == 1 else 1)

In [83]:
np.unique(Y, return_counts=True)

(array([0, 1]), array([221,   9]))

### ensembles

In [91]:
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier

In [95]:
clf = SVC(**{'C': 10, 'class_weight': 'balanced', 'degree': 3, 'gamma': 'auto', 'kernel': 'sigmoid', 'probability': True})
ens_clf = BaggingClassifier(clf)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
ens_clf.fit(X_train, y_train)
ens_clf.predict(X_test)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [96]:
clf = SVC(**{'C': 10, 'class_weight': 'balanced', 'degree': 3, 'gamma': 'auto', 'kernel': 'sigmoid', 'probability': True})
ens_clf = AdaBoostClassifier(clf)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
ens_clf.fit(X_train, y_train)
ens_clf.predict(X_test)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### anomaly detection

In [18]:
from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer

In [45]:
ad_clf = OneClassSVM(kernel="rbf")
scores = cross_val_score(ad_clf, X, [_ if _ == 1 else -1 for _ in Y], cv=k_fold, scoring=make_scorer(accuracy_score))
print(scores)
print(np.average(scores))

[0.51282051 0.43589744 0.71052632 0.39473684 0.55263158 0.34210526]
0.4914529914529915


In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
ad_clf = OneClassSVM(nu=0.7)
ad_clf.fit(X_train, y_train)
y_predict = ad_clf.predict(X_test)
print(accuracy_score([x if x == 1 else -1 for x in y_test], y_predict))
print(y_predict)

0.6578947368421053
[ 1 -1 -1 -1 -1 -1 -1  1 -1  1 -1  1 -1 -1  1 -1 -1 -1  1 -1 -1 -1 -1 -1
 -1 -1  1 -1 -1 -1 -1 -1  1  1 -1 -1 -1  1  1 -1  1 -1  1 -1 -1  1 -1 -1
 -1 -1 -1 -1  1 -1  1  1  1 -1  1 -1  1 -1 -1  1 -1 -1  1 -1 -1  1 -1 -1
  1 -1 -1  1]


In [49]:
from sklearn.model_selection import GridSearchCV

In [69]:
param_grid = [
  {'nu': np.arange(.1, 1.0, 0.1), 'gamma': ['auto'], 'kernel': ['rbf']},
 ]
gs_cv = GridSearchCV(OneClassSVM(), param_grid=param_grid, scoring=make_scorer(accuracy_score), cv=5, refit=True)
gs_cv.fit(X, [_ if _ == 1 else -1 for _ in Y])

GridSearchCV(cv=5, error_score='raise',
       estimator=OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='auto', kernel='rbf',
      max_iter=-1, nu=0.5, random_state=None, shrinking=True, tol=0.001,
      verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'nu': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]), 'gamma': ['auto'], 'kernel': ['rbf']}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(accuracy_score), verbose=0)

In [70]:
import pandas as pd
pd.DataFrame(gs_cv.cv_results_)

/Users/sahil/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/sahil/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/sahil/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/sahil/anaconda3/lib/python

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_gamma,param_kernel,param_nu,params,rank_test_score,split0_test_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,0.005670,0.001463,0.356522,0.215217,auto,rbf,0.1,"{'gamma': 'auto', 'kernel': 'rbf', 'nu': 0.1}",9,0.326087,...,0.521739,0.211957,0.326087,0.217391,0.282609,0.184783,0.000423,0.000202,0.084307,0.017728
1,0.005984,0.001430,0.365217,0.220652,auto,rbf,0.2,"{'gamma': 'auto', 'kernel': 'rbf', 'nu': 0.2}",8,0.326087,...,0.565217,0.239130,0.326087,0.271739,0.282609,0.173913,0.000636,0.000207,0.101408,0.033958
2,0.006545,0.001272,0.395652,0.317391,auto,rbf,0.3,"{'gamma': 'auto', 'kernel': 'rbf', 'nu': 0.300...",7,0.326087,...,0.565217,0.331522,0.369565,0.298913,0.282609,0.320652,0.000886,0.000129,0.098572,0.015975
3,0.007676,0.001584,0.430435,0.389130,auto,rbf,0.4,"{'gamma': 'auto', 'kernel': 'rbf', 'nu': 0.4}",6,0.369565,...,0.586957,0.369565,0.413043,0.385870,0.304348,0.407609,0.000599,0.000195,0.096635,0.012676
4,0.008341,0.001964,0.482609,0.483696,auto,rbf,0.5,"{'gamma': 'auto', 'kernel': 'rbf', 'nu': 0.5}",5,0.478261,...,0.608696,0.500000,0.434783,0.483696,0.413043,0.478261,0.000598,0.000230,0.067915,0.009094
5,0.009391,0.002086,0.578261,0.571739,auto,rbf,0.6,"{'gamma': 'auto', 'kernel': 'rbf', 'nu': 0.6}",4,0.521739,...,0.673913,0.576087,0.500000,0.570652,0.586957,0.570652,0.000747,0.000212,0.062403,0.004067
6,0.009402,0.001962,0.669565,0.667391,auto,rbf,0.7,"{'gamma': 'auto', 'kernel': 'rbf', 'nu': 0.700...",3,0.608696,...,0.739130,0.668478,0.586957,0.673913,0.630435,0.663043,0.000820,0.000088,0.077044,0.006338
7,0.010729,0.002397,0.782609,0.771739,auto,rbf,0.8,"{'gamma': 'auto', 'kernel': 'rbf', 'nu': 0.8}",2,0.695652,...,0.804348,0.777174,0.782609,0.777174,0.804348,0.777174,0.000792,0.000244,0.045600,0.006875
8,0.010853,0.002818,0.873913,0.869565,auto,rbf,0.9,"{'gamma': 'auto', 'kernel': 'rbf', 'nu': 0.9}",1,0.869565,...,0.913043,0.869565,0.826087,0.875000,0.869565,0.864130,0.000256,0.000363,0.028840,0.004861


In [71]:
gs_cv.best_score_

0.8739130434782608

In [68]:
gs_cv.best_params_

{'degree': 6, 'gamma': 'auto', 'kernel': 'poly', 'nu': 0.4}

### normal svm and kernel selection

In [7]:
k_fold = 6

In [8]:
kernel = 'poly'
print('Kernel: ', kernel)
clf = SVC(kernel=kernel, class_weight='balanced')
np.average(cross_val_score(clf, X, Y, cv=k_fold))

Kernel:  poly


0.9610825400299084

In [9]:
kernel = 'rbf'
print('Kernel: ', kernel)
clf = SVC(kernel=kernel, class_weight='balanced')
np.average(cross_val_score(clf, X, Y, cv=k_fold))

Kernel:  rbf


0.9482620272093957

In [10]:
kernel = 'sigmoid'
print('Kernel: ', kernel)
clf = SVC(kernel=kernel, class_weight='balanced')
np.average(cross_val_score(clf, X, Y, cv=k_fold))

Kernel:  sigmoid


0.9002230975915185

In [11]:
# kernel = 'precomputed'
# print('Kernel: ', kernel)
# clf = SVC(kernel=kernel, class_weight='balanced')
# cross_val_score(clf, X, Y, cv=k_fold)

In [149]:
# n_samples / (n_classes * np.bincount(y))
X.shape[0] / (2*np.bincount(Y))

array([  0.52036199,  12.77777778])

In [86]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
clf = SVC(kernel='poly', class_weight='balanced', probability=True)
clf.fit(X_train, y_train)
print(clf.predict(X_test))
clf.predict_proba(X_test)[:,1]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0]


array([0.0415634 , 0.04365995, 0.00182686, 0.04664141, 0.04411105,
       0.04331738, 0.0484596 , 0.00127521, 0.04523435, 0.0435793 ,
       0.04427737, 0.04421218, 0.0440134 , 0.04514256, 0.04492044,
       0.04163024, 0.01415224, 0.04695233, 0.04320325, 0.04420214,
       0.04390487, 0.04575226, 0.0464508 , 0.04284476, 0.04640284,
       0.04522823, 0.04429313, 0.04635346, 0.04310751, 0.04565618,
       0.04426294, 0.02688375, 0.03886473, 0.04365537, 0.04585414,
       0.04366811, 0.04500299, 0.04460703, 0.04352903, 0.04406892,
       0.04654779, 0.04447094, 0.04201015, 0.04326055, 0.04281197,
       0.0433026 , 0.04371573, 0.04417434, 0.04351211, 0.04698533,
       0.04435703, 0.04658198, 0.04442717, 0.04319824, 0.04359141,
       0.04380029, 0.04465192, 0.04487563, 0.04539705, 0.01931896,
       0.04345568, 0.04326056, 0.04322224, 0.04370086, 0.04355741,
       0.04479891, 0.04204287, 0.04371642, 0.04535695, 0.04569559,
       0.04160463, 0.04522647, 0.04433792, 0.04536438, 0.04332

In [81]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
clf = SVC(kernel='poly', class_weight='balanced')
clf.fit(X_train, y_train)
clf.predict(X_test)
# clf.score(X_test, y_test)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [195]:
p = PCA(n_components=160)
p.fit(X)

percents = np.round((p.explained_variance_ratio_), 3)
_sum = 0
# print(np.sum(percents))
j = 0
for i in range(21):
    s = np.sum(percents[j: j+5])*100
    _sum += s
    print(j, s, _sum)
    j += 5


0 14.0 14.0
5 9.4 23.4
10 7.9 31.3
15 6.3 37.6
20 5.4 43.0
25 4.8 47.8
30 4.3 52.1
35 4.0 56.1
40 3.6 59.7
45 3.5 63.2
50 3.1 66.3
55 3.0 69.3
60 2.8 72.1
65 2.5 74.6
70 2.5 77.1
75 2.5 79.6
80 2.1 81.7
85 2.0 83.7
90 2.0 85.7
95 2.0 87.7
100 2.0 89.7


In [190]:
pca = PCA(n_components=60)
pca.fit(X)
print(np.round((pca.explained_variance_ratio_), 3))

# pca.components_
# ['PC-1','PC-2','PC-3','PC-4','PC-5','PC-6']

coef = pca.transform(np.eye(X.shape[1]))
print(np.linalg.norm(coef, axis=0))

_p = pd.DataFrame(coef, columns=range(1, 61), index=og_X.columns)

abs(_p).idxmax()

# pd.value_counts(abs(_p).idxmax(axis=1))

# clf = SVC(kernel='poly', class_weight='balanced')
# X_train, X_test, y_train, y_test = train_test_split(pca.transform(X), Y, test_size=0.33, random_state=42)

# clf.fit(X_train, y_train)

# clf.predict(X_test), clf.score(X_test, y_test)


# # cross_val_score(clf, pca.transform(X), Y, cv=k_fold)

[ 0.046  0.029  0.023  0.021  0.021  0.02   0.019  0.019  0.018  0.018
  0.017  0.016  0.016  0.015  0.015  0.014  0.013  0.012  0.012  0.012
  0.011  0.011  0.011  0.011  0.01   0.01   0.01   0.01   0.009  0.009
  0.009  0.009  0.009  0.008  0.008  0.008  0.008  0.008  0.008  0.008
  0.008  0.007  0.007  0.007  0.007  0.007  0.007  0.007  0.007  0.007
  0.007  0.006  0.006  0.006  0.006  0.006  0.006  0.006  0.006  0.006]
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.]


1                            average_outflow
2                             utm_medium_ppc
3                  firm_type_Private Limited
4                      campaign_city_Kolkata
5                           state_TAMIL NADU
6     registered_office_state_ANDHRA PRADESH
7                               city_Chennai
8     registered_office_state_MADHYA PRADESH
9          registered_office_state_KARNATAKA
10           registered_office_state_HARYANA
11                               city_Indore
12                      state_MADHYA PRADESH
13           registered_office_state_GUJARAT
14      registered_office_state_CHHATTISGARH
15                           state_KARNATAKA
16         registered_office_state_TELANGANA
17                     platform_mobile:Linux
18            registered_office_state_PUNJAB
19                          role_in_firm_2.0
20               registered_office_city_DHAR
21                               city_Kalyan
22                            industry_132.0
23        

In [66]:
pca

PCA(copy=True, iterated_power='auto', n_components=6, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)